In [1]:
1+1

2

In [2]:
import mozo2 

import importlib
importlib.reload(mozo2)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mozo import compare_and_combine_2, Diff, merge_with_features, num_to_range_categories
import pandas as pd
import numpy as np
np.random.seed(0)

import xgboost as xgb

from scipy import stats
import math 
import datetime 

from sklearn.metrics import r2_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.svm import SVC

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime


In [3]:
total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201101_07_20.h5.csv')
total.info(verbose=True)

/home/conda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,86,161) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 218 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   breadcrumb                                              object 
 1   created_at                                              object 
 2   price_currency                                          object 
 3   price_details                                           object 
 4   price_value                                             float64
 5   seller_address                                          object 
 6   seller_name                                             object 
 7   seller_type                                             object 
 8   feature_czujniki_parkowania_przednie                    bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana            bool   
 10  feature_kurtyny_powietrzne                             

print_plots_by_type(total, type =[np.object])

In [4]:
## modelowanie

In [4]:
train_30 = pd.read_hdf('output/train_30.h5', 'data') 
test_car_ids = train_30['car_id'].values

train_70 = total[~total['car_id'].isin(test_car_ids) & ~total['price_value'].isna()]
train_30 = total[total['car_id'].isin(test_car_ids)]

print(len(total.index))
print(len(total[~total['price_value'].isna()].index))

print(train_70['price_value'].isna().sum())
print(train_30['price_value'].isna().sum())

train_100 = total[~total['price_value'].isna()]
print(train_100['price_value'].isna().sum())

global_min = total[~total['price_value'].isna()].price_value.min()

test = total[total['price_value'].isna()]
train = total[~total['price_value'].isna()]


160616
106494
0
0
0


In [8]:
start_vars = np.sort([

# 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_hud__wyświetlacz_przezierny_', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tapicerka_welurowa', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'latitude_na_mean', 'longitude', 'longitude_na_mean', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2'
# 'model_8790', 'model_XGBRegressor_2608_37590_po_hiperopcie', 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_cd', 'feature_centralny_zamek', 'feature_czujnik_martwego_pola', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'longitude_na_mean', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_akryl__niemetalizowany_', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_cd', 'feature_centralny_zamek', 'feature_czujnik_martwego_pola', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'longitude_na_mean', 'model_8790', 'model_XGBRegressor_3293_22097_', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_akryl__niemetalizowany_', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
])

# 'model_8790' = car_price_log_Vladimir_categories_xgb.csv; 5637.56722; "opis": "Vladimir z kategorialnymi mae na train_30"

black_list = ['car_id', 'price_details', 'price_value', 'price_value_log', 'price_value_pln', 'price_value_pln_log_transl', 'new_cat_price_value',
#               *[x for x in total.columns if x.startswith('model_')],
              'model_XGBRegressor_2195_24982_po_hiperopcie_caly_train', #train_100, przeuczony
              'model_XGBRegressor_3293_22097_', #train_100
              'longitude_na_mean', 'latitude_na_mean', #gorsze w xbfir
              *total.columns[total.isnull().any()],
             ]

feats3 = total.select_dtypes(include=[np.number, np.bool]).columns
feats3 = [x for x in feats3 if x not in black_list]
feats3 = np.sort(feats3)
print(total[feats3].info(verbose=True))
print(total[feats3].columns[total[feats3].isnull().any()])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 163 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   created_at_unix                                     float64
 1   created_at_unix_na_mean                             float64
 2   encoded_param_kolor_2                               int64  
 3   encoded_param_napęd_2                               int64  
 4   encoded_param_rodzaj_paliwa_2                       int64  
 5   encoded_param_skrzynia_biegów_2                     int64  
 6   encoded_param_typ_2                                 int64  
 7   encoded_seller_miasto_297                           int64  
 8   feature_abs                                         bool   
 9   feature_alarm                                       bool   
 10  feature_alufelgi                                    bool   
 11  feature_asr__kontrola_trakcji_        

In [9]:
import contextlib
import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor
import json

# params = {                                                                                     
#     "iterations": 704,
#     "depth": 6,
#     "learning_rate": 0.23701660211398967,
#     "random_strength": 0.13419513355474952,
#     "bagging_temperature": 0.29043879590002025,
#     "border_count": 78,
#     "l2_leaf_reg": 10,
#     "random_state": 6950,
#     "silent": True
# }
params = {                                                                                     
    "iterations": 200,
    "depth": 8,
#     "learning_rate": 0.23701660211398967,
#     "random_strength": 0.13419513355474952,
#     "bagging_temperature": 0.29043879590002025,
#     "border_count": 78,
#     "l2_leaf_reg": 10,
    "random_state": 6950,
    "silent": True
}


mae_global_min = 2778.115628209074 
DELTA = 500.0

big_iter = 0

current_vars=start_vars
all_vars = feats3 #maksymalny set

while True:

    pairs_so_far = []

    last_added = None
    last_removed = None

    last_trial = False
    big_iter += 1
    print("START iter=" + str(big_iter) + ', mae_global_min=' + str(mae_global_min))
    
    while True:
        with contextlib.redirect_stdout(None):
            current_vars, last_added, last_removed, pairs_so_far, last_trial =  mozo2.mutate_rand_feature_pairs(current_vars, all_vars, pairs_so_far)
#         print('pairs_so_far: ' + json.dumps(pairs_so_far, ensure_ascii=False, indent=4))
        print('added: ' + last_added)
        print('removed: ' + last_removed)
        print('len: ' + str(len(current_vars)))
        
        mae, r2, model, model_cl, learning_curve_plot = mozo2.train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
                     current_vars, global_min, 
                     CatBoostRegressor, params, digitize=None, subfolder='CatBoostRegressor_parami_delta',
                     file_sufix = 'parami', opis='start=2778.115628209074', 
                     learning_curve = False, 
                     add_model_column_min = -1, total = total, 
                     save_min=mae_global_min, kaggle_min=-1)

        if mae < mae_global_min + DELTA:
            if (mae < mae_global_min):
                mae_global_min = mae
                print('Improvement: ' + last_removed + ' -> ' + last_added)
            else:
                print('Mutation: ' + last_removed + ' -> ' + last_added)
        else:
            current_vars = mozo2.replace_feature(current_vars, last_added, last_removed)
            print('Rollback: ' + last_added + ' -> ' + last_removed)
        print('=============================================' + str(mae_global_min)+ '=============================================')

        if last_trial == True: 
            print("END iter=" + str(big_iter) + ', mae_global_min=' + str(mae))
            break

START iter=1, mae_global_min=2778.115628209074
added: param_faktura_vat
removed: feature_gniazdo_sd
len: 51
mea: 2776.816641956884
r2: 0.9803750856197057
Improvement: feature_gniazdo_sd -> param_faktura_vat
=============================================2776.816641956884=============================================
added: param_pierwsza_rejestracja_date_na
removed: new_param_kolor_cat
len: 51
mea: 2790.7306269853193
r2: 0.9799880463716715
Mutation: new_param_kolor_cat -> param_pierwsza_rejestracja_date_na
=============================================2776.816641956884=============================================
added: param_miesięczna_rata_2
removed: longitude
len: 51
mea: 2828.292211670131
r2: 0.9786527059170903
Mutation: longitude -> param_miesięczna_rata_2
=============================================2776.816641956884=============================================
added: feature_kamera_cofania
removed: feature_podgrzewane_tylne_siedzenia
len: 51
mea: 2820.74281492476
r2: 0.9788418357529

mea: 2844.080118107546
r2: 0.9787598187542024
Mutation: feature_elektrycznie_ustawiane_lusterka -> feature_czujnik_zmierzchu
=============================================2776.816641956884=============================================
added: param_metalik
removed: feature_czujnik_zmierzchu
len: 51
mea: 2865.875200088122
r2: 0.9797022821702848
Mutation: feature_czujnik_zmierzchu -> param_metalik
=============================================2776.816641956884=============================================
added: feature_radio_fabryczne
removed: new_seller_type_cat
len: 51
mea: 2865.9504992172724
r2: 0.9784063281981452
Mutation: new_seller_type_cat -> feature_radio_fabryczne
=============================================2776.816641956884=============================================
added: feature_czujniki_parkowania_tylne
removed: param_metalik
len: 51
mea: 2887.3701702226444
r2: 0.9786487284048702
Mutation: param_metalik -> feature_czujniki_parkowania_tylne
====================================

mea: 2882.8092202299244
r2: 0.9794008893015246
Mutation: param_kierownica_po_prawej__anglik_ -> param_rok_produkcji_poprzedni_rok
=============================================2776.816641956884=============================================
added: new_param_model_pojazdu_cat
removed: param_pojemność_skokowa
len: 51
mea: 2880.2135448930603
r2: 0.9787712183115714
Mutation: param_pojemność_skokowa -> new_param_model_pojazdu_cat
=============================================2776.816641956884=============================================
added: new_param_skrzynia_biegów_cat
removed: param_filtr_cząstek_stałych
len: 51
mea: 2883.1223997168536
r2: 0.9788851799302909
Mutation: param_filtr_cząstek_stałych -> new_param_skrzynia_biegów_cat
=============================================2776.816641956884=============================================
added: feature_tuner_tv
removed: feature_system_start_stop
len: 51
mea: 2861.0082189115938
r2: 0.9790663228155532
Mutation: feature_system_start_stop -> featu

mea: 2829.6188541099796
r2: 0.978125489439135
Mutation: param_wartość_wykupu_2 -> feature_kurtyny_powietrzne
=============================================2776.816641956884=============================================
added: feature_wielofunkcyjna_kierownica
removed: new_param_emisja_co2_cat
len: 51
mea: 2847.0571119016777
r2: 0.9778574193323717
Mutation: new_param_emisja_co2_cat -> feature_wielofunkcyjna_kierownica
=============================================2776.816641956884=============================================
added: feature_klimatyzacja_dwustrefowa
removed: feature_światła_led
len: 51
mea: 2867.1687360849633
r2: 0.9766898949249251
Mutation: feature_światła_led -> feature_klimatyzacja_dwustrefowa
=============================================2776.816641956884=============================================
added: param_pierwszy_właściciel
removed: feature_tapicerka_skórzana
len: 51
mea: 2863.1543164259915
r2: 0.9769005103749197
Mutation: feature_tapicerka_skórzana -> param_pierw

mea: 2987.9635529334196
r2: 0.9725923749102127
Mutation: param_tuning -> param_metalik
=============================================2776.816641956884=============================================
added: new_param_rodzaj_paliwa_cat
removed: new_vin_plant_cat
len: 51
mea: 2985.2996361560604
r2: 0.9725934883532146
Mutation: new_vin_plant_cat -> new_param_rodzaj_paliwa_cat
=============================================2776.816641956884=============================================
added: feature_asr__kontrola_trakcji_
removed: feature_dach_panoramiczny
len: 51
mea: 2987.2194352347424
r2: 0.9725610201095346
Mutation: feature_dach_panoramiczny -> feature_asr__kontrola_trakcji_
=============================================2776.816641956884=============================================
added: param_zarejestrowany_jako_zabytek
removed: ogloszenie_dni_wisi
len: 51
mea: 3004.9490021976458
r2: 0.9723726892259611
Mutation: ogloszenie_dni_wisi -> param_zarejestrowany_jako_zabytek
=======================

...

mea: 2866.1523915472985
r2: 0.9741808450010172
Mutation: feature_podgrzewane_lusterka_boczne -> feature_poduszka_powietrzna_kierowcy
=============================================2776.816641956884=============================================
added: created_at_unix_na_mean
removed: feature_poduszka_powietrzna_kierowcy
len: 51
mea: 2904.9378879541223
r2: 0.9739050524481137
Mutation: feature_poduszka_powietrzna_kierowcy -> created_at_unix_na_mean
=============================================2776.816641956884=============================================
added: param_rok_produkcji_ten_rok
removed: new_cat_seller_address
len: 51
mea: 2901.558913109792
r2: 0.9743156527248095
Mutation: new_cat_seller_address -> param_rok_produkcji_ten_rok
=============================================2776.816641956884=============================================
added: param_vat_marża
removed: feature_system_start_stop
len: 51
mea: 2888.769188349216
r2: 0.9741713337845671
Mutation: feature_system_start_stop -> p

mea: 2884.082629928536
r2: 0.9733771093913255
Mutation: feature_asystent_pasa_ruchu -> longitude
=============================================2776.816641956884=============================================
added: model_RandomForestRegressor_7089_27166_feats2
removed: ogloszenie_miesiac
len: 51
mea: 5554.688028620302
r2: 0.9446619873805602
Rollback: model_RandomForestRegressor_7089_27166_feats2 -> ogloszenie_miesiac
=============================================2776.816641956884=============================================
added: feature_podgrzewane_lusterka_boczne
removed: param_pierwsza_rejestracja_ten_rok
len: 51
mea: 2889.890545494744
r2: 0.9731480035857509
Mutation: param_pierwsza_rejestracja_ten_rok -> feature_podgrzewane_lusterka_boczne
=============================================2776.816641956884=============================================
added: model_XGBRegressor_3422_09438_start_vars_train_100
removed: param_pojemność_skokowa
len: 51
mea: 2934.2361259197332
r2: 0.977271044207

mea: 3000.446097975644
r2: 0.9784231314647113
Mutation: new_param_skrzynia_biegów_cat -> param_perłowy
=============================================2776.816641956884=============================================
added: feature_radio_niefabryczne
removed: param_matowy
len: 51
mea: 3005.845488166423
r2: 0.9777843818952225
Mutation: param_matowy -> feature_radio_niefabryczne
=============================================2776.816641956884=============================================
added: feature_tapicerka_welurowa
removed: feature_kurtyny_powietrzne
len: 51
mea: 2980.333751649961
r2: 0.9782128651126037
Mutation: feature_kurtyny_powietrzne -> feature_tapicerka_welurowa
=============================================2776.816641956884=============================================
added: new_param_liczba_pozostałych_rat_cat
removed: encoded_param_skrzynia_biegów_2
len: 51
mea: 3003.512970702327
r2: 0.9764163845533802
Mutation: encoded_param_skrzynia_biegów_2 -> new_param_liczba_pozostałych_rat_ca

mea: 5634.70429233063
r2: 0.9430110133331142
Rollback: model_RandomForestRegressor_7089_27166_feats2 -> new_seller_name_cat
=============================================2776.816641956884=============================================
added: param_vat_free
removed: param_tuning
len: 51
mea: 2869.4431509233636
r2: 0.9732369472147862
Mutation: param_tuning -> param_vat_free
=============================================2776.816641956884=============================================
added: feature_wspomaganie_kierownicy
removed: model_8790
len: 51
mea: 12254.33481330937
r2: 0.7837691437055269
Rollback: feature_wspomaganie_kierownicy -> model_8790
=============================================2776.816641956884=============================================
added: ogloszenie_miesiac
removed: new_seller_wojewodzctwo_cat
len: 51
mea: 2883.3113439016797
r2: 0.9727393605183889
Mutation: new_seller_wojewodzctwo_cat -> ogloszenie_miesiac
=============================================2776.816641956884=====

KeyboardInterrupt: 

In [21]:
model = RandomForestRegressor(n_estimators=50, max_depth=7).fit(X_sub_train, y_sub_train) 
perm = PermutationImportance(model).fit(X_sub_test, y_sub_test) 
eli5.show_weights(perm, feature_names=random_feats, top=50) 


NameError: name 'RandomForestRegressor' is not defined